<a href="https://colab.research.google.com/github/Yukthi231/registry/blob/main/train_chatbot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
import nltk

from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()
import json
import pickle
import os

In [5]:
import numpy as np
from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout
from tensorflow.keras.optimizers import SGD
import random

In [6]:
words=[]
classes = []
documents = []
ignore_words = ['?', '!']
data_file = open('intents.json').read()
intents = json.loads(data_file)

In [7]:
files = [f for f in os.listdir('.') if os.path.isfile(f)]
files

['classes.pkl', 'words.pkl', 'intents.json']

In [8]:
nltk.download('punkt')
for intent in intents['intents']:
    for pattern in intent['patterns']:

        #tokenize each word
        print(pattern)
        w = nltk.word_tokenize(pattern)
        words.extend(w)
        #add documents in the corpus
        documents.append((w, intent['tag']))

        # add to our classes list
        if intent['tag'] not in classes:
            classes.append(intent['tag'])

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
Hi there
How are you
Is anyone there?
Hey
Hola
Hello
Good day
Bye
See you later
Goodbye
Nice chatting to you, bye
Till next time
Thanks
Thank you
That's helpful
Awesome, thanks
Thanks for helping me
How you could help me?
What you can do?
What help you provide?
How you can be helpful?
What support is offered
How to check Adverse drug reaction?
Open adverse drugs module
Give me a list of drugs causing adverse behavior
List all drugs suitable for patient with adverse reaction
Which drugs dont have adverse reaction?
Open blood pressure module
Task related to blood pressure
Blood pressure data entry
I want to log blood pressure results
Blood pressure data management
I want to search for blood pressure result history
Blood pressure for patient
Load patient blood pressure result
Show blood pressure results for patient
Find blood pressure results by ID
Find me a pharmacy
Find pharmacy
Lis

In [9]:

# lemmaztize and lower each word and remove duplicates
nltk.download('wordnet')
words = [lemmatizer.lemmatize(w.lower()) for w in words if w not in ignore_words]
words = sorted(list(set(words)))

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


In [10]:
# sort classes
classes = sorted(list(set(classes)))

In [11]:
# documents = combination between patterns and intents
print (len(documents), "documents")

47 documents


In [12]:
# classes = intents
print (len(classes), "classes", classes)

9 classes ['adverse_drug', 'blood_pressure', 'blood_pressure_search', 'goodbye', 'greeting', 'hospital_search', 'options', 'pharmacy_search', 'thanks']


In [13]:
# words = all words, vocabulary
print (len(words), "unique lemmatized words", words)

88 unique lemmatized words ["'s", ',', 'a', 'adverse', 'all', 'anyone', 'are', 'awesome', 'be', 'behavior', 'blood', 'by', 'bye', 'can', 'causing', 'chatting', 'check', 'could', 'data', 'day', 'detail', 'do', 'dont', 'drug', 'entry', 'find', 'for', 'give', 'good', 'goodbye', 'have', 'hello', 'help', 'helpful', 'helping', 'hey', 'hi', 'history', 'hola', 'hospital', 'how', 'i', 'id', 'is', 'later', 'list', 'load', 'locate', 'log', 'looking', 'lookup', 'management', 'me', 'module', 'nearby', 'next', 'nice', 'of', 'offered', 'open', 'patient', 'pharmacy', 'pressure', 'provide', 'reaction', 'related', 'result', 'search', 'searching', 'see', 'show', 'suitable', 'support', 'task', 'thank', 'thanks', 'that', 'there', 'till', 'time', 'to', 'transfer', 'up', 'want', 'what', 'which', 'with', 'you']


In [14]:
pickle.dump(words,open('words.pkl','wb'))
pickle.dump(classes,open('classes.pkl','wb'))

In [15]:
# create our training data
training = []

In [16]:
# create an empty array for our output
output_empty = [0] * len(classes)

In [17]:
# training set, bag of words for each sentence
for doc in documents:
    # initialize our bag of words
    bag = []
    # list of tokenized words for the pattern
    pattern_words = doc[0]
    # lemmatize each word - create base word, in attempt to represent related words
    pattern_words = [lemmatizer.lemmatize(word.lower()) for word in pattern_words]

In [18]:
 # create our bag of words array with 1, if word match found in current pattern
for w in words:
    bag.append(1) if w in pattern_words else bag.append(0)

In [19]:
# output is a '0' for each tag and '1' for current tag (for each pattern)
output_row = list(output_empty)
output_row[classes.index(doc[1])] = 1
    
training.append([bag, output_row])


In [20]:
# shuffle our features and turn into np.array
random.shuffle(training)
training = np.array(training)
np.append(training,[bag, output_row])

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  This is separate from the ipykernel package so we can avoid doing imports until
/usr/local/lib/python3.7/dist-packages/numpy/core/_asarray.py:136: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  return array(a, dtype, copy=False, order=order, subok=True)


array([list([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0]),
       list([0, 0, 0, 0, 0, 1, 0, 0, 0]),
       list([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0]),
       list([0, 0, 0, 0, 0, 1, 0, 0, 0])], dtype=object)

In [21]:
# shuffle our features and turn into np.array
random.shuffle(training)
training = np.array(training)
# create train and test lists. X - patterns, Y - intents
train_x = list(training[:,0])
train_y = list(training[:,1])
print("Training data created")

Training data created


In [22]:
# Create model - 3 layers. First layer 128 neurons, second layer 64 neurons and 3rd output layer contains number of neurons
# equal to number of intents to predict output intent with softmax
model = Sequential()
model.add(Dense(128, input_shape=(len(train_x[0]),), activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(len(train_y[0]), activation='softmax'))

In [23]:
# Compile model. Stochastic gradient descent with Nesterov accelerated gradient gives good results for this model
sgd = SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True)
model.compile(loss='categorical_crossentropy', optimizer=sgd, metrics=['accuracy'])

/usr/local/lib/python3.7/dist-packages/keras/optimizer_v2/optimizer_v2.py:356: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  "The `lr` argument is deprecated, use `learning_rate` instead.")


In [24]:
#fitting and saving the model 
hist = model.fit(np.array(train_x), np.array(train_y), epochs=200, batch_size=5, verbose=1)
model.save('chatbot_model.h5', hist)


Epoch 1/200
1/1 [==============================] - 1s 808ms/step - loss: 1.7998 - accuracy: 0.0000e+00
Epoch 2/200
1/1 [==============================] - 0s 5ms/step - loss: 1.7832 - accuracy: 1.0000
Epoch 3/200
1/1 [==============================] - 0s 5ms/step - loss: 1.5416 - accuracy: 1.0000
Epoch 4/200
1/1 [==============================] - 0s 6ms/step - loss: 1.4941 - accuracy: 0.0000e+00
Epoch 5/200
1/1 [==============================] - 0s 7ms/step - loss: 1.6345 - accuracy: 1.0000
Epoch 6/200
1/1 [==============================] - 0s 14ms/step - loss: 0.9846 - accuracy: 1.0000
Epoch 7/200
1/1 [==============================] - 0s 6ms/step - loss: 1.3187 - accuracy: 1.0000
Epoch 8/200
1/1 [==============================] - 0s 7ms/step - loss: 0.5796 - accuracy: 1.0000
Epoch 9/200
1/1 [==============================] - 0s 6ms/step - loss: 0.4253 - accuracy: 1.0000
Epoch 10/200
1/1 [==============================] - 0s 5ms/step - loss: 0.6374 - accuracy: 1.0000
Epoch 11/200
1/1 [

In [25]:
print("model created")

model created
